**Prediction of the next word by the preceeding 8 words**    

Update 4/12/2021 by Shogo Ohmae     
1.  Removed word2vec embedding     
2.  Added a layer name for the RNN.  layers.SimpleRNN(HIDDEN_DIM, return_state=True, name='sRNN')     

Update 4/22/2021 by Shogo Ohmae  
3.   Saved learning curve, with larger epoch num in model.fit.    
     //80,  in steps_per_epoch=(train_end - train_start) //80 // batch_size,    
4.   smaller learning rate: 0.002 <- 0.01, with slower decay rate: 0.9          
5.   monitoring learning curve with top 5 accuracy:    
     metrics=[keras.metrics.TopKCategoricalAccuracy(k=5)]    
6.   Suffle sentenses, to uniform the differences of context in different books, to make contexts of validation and training dataset uniform.    

Update 4/24/2021 by Shogo Ohmae  
7.   256 cells did not improve the performance.    

Update 4/28/2021 by Shogo Ohmae  
8.   Hyper-parameter tuning:  lr:0.004, decay_rate = 0.9, patience=increments//4,    
9.   Optimizer tuning:  AdaDelta was bad. Adam is better than RMSprop. 


next:    
x axis of lerning curve (sentenses)   

cut later part of sentenses, before word dict
Data set selection by parser, evaluation with SVO index by parser,
learning in neocortex   

In [ ]:

ver = 2099;  # custom loss,  128 fb cells,   16 time steps @220117, 8 ep, batch =16

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')
# %cd "gdrive/My Drive/Colab/NLP" 

In [ ]:
import numpy as np
np.set_printoptions(threshold=np.inf)

In [ ]:
!pip install gensim
from gensim.models import KeyedVectors

In [ ]:
# !wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

In [ ]:
# ##### Loading word2vec #####
# EMBEDDING_FILE = '/root/input/GoogleNews-vectors-negative300.bin.gz' 
# word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import preprocessing

In [ ]:
print(tf.__version__)

2.8.0


In [ ]:
#### load text data file #####
import nltk

In [ ]:
nltk.download('gutenberg')
# nltk.corpus.gutenberg.fileids()
text_=[]
text_.append( nltk.corpus.gutenberg.raw('carroll-alice.txt') )
text_.append( nltk.corpus.gutenberg.raw('austen-emma.txt') )
text_.append( nltk.corpus.gutenberg.raw('austen-persuasion.txt') )
text_.append( nltk.corpus.gutenberg.raw('austen-sense.txt') )
text_.append( nltk.corpus.gutenberg.raw('melville-moby_dick.txt') )
text_.append( nltk.corpus.gutenberg.raw('burgess-busterbrown.txt') ) # children's book, "The Adventures of Buster Bear" "Fun with Farmer Brown's Boy"
text_.append( nltk.corpus.gutenberg.raw('edgeworth-parents.txt') ) # children's book, "The Parent's Assistant", 1796 
text_br = nltk.corpus.gutenberg.raw('bryant-stories.txt')  # "Stories to Tell to Children", by Sara Cone Bryant 1918

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [ ]:
# nltk.corpus.gutenberg.fileids()

In [ ]:
txt_1 = """My name is Anne. I am a Japanese high school student. I came to live in America with my family two weeks ago. One day Mother asked me about my school life. I answered that I enjoyed it very much. But that was not true. I didn't want Mother to worry about me. I always said to myself, "I live in two worlds, one at home and the other at school. They are very different. At home I speak Japanese and live a happy Japanese life with my family. I feel that school is far away when I am at home. In class at school teachers sometimes speak English so fast that I don't understand them. I want to talk with a friend about that, but I don't have any friends. Every day I wait for a student who will talk to me." The next day in art class the teacher stopped at my desk and said, "Akiko, your picture is very wonderful! " Then many students came to see my picture. After the art class one girl student came to me. At last a student talked to me! She said, "Hi, Anne. I'm Suzzan. I would like to talk with you after school." I was very happy. I said, "Yes, of course." After school we went to Mary's house and began to talk. Mary was also interested in drawing pictures. We talked about a lot of things. When I talked about my idea of the two worlds, Mary said, "I didn't know that. But you can ask teachers to speak more slowly. If you want to make your school life happy, you should try to say something to us. We will help you any time." I had a very good time with Mary. I ran home to tell Mother about my new friend, but she was not there. Soon Mother came home. "Mother, where have you been? "I asked. "I have been to English school. I began to learn English, " answered Mother. "You don't have to learn English so hard because you stay home every day, " I said. "I want to make friends to learn about America and tell them a lot about Japan, " said Mother. "Mary and Mother have the same idea, " I thought. " Today I learned an important thing from Mary and Mother. It is not good to stay in our small world when we are in a foreign culture. I will try hard to make my school life happy."
Relearning the art of seeing the world around us is quite simple, although it takes practice and requires breaking some bad abits. And "relearn" is the correct word. Most of us observed much more as children than we do as adults. A child's day is filled with newness and wonder. The desire to explore, to have an adventure, gave us all a natural ability to be aware of the world around us. 
But as adults we are slow to be stimulated by new ideas and new situations. We fail to see the wonder of the things around us. The first step in awakening our senses is to rediscover the ability to observe that we had as children. To do so we need to stop anticipating what we are going to see and feel before it happens. Such anticipation blocks our ability to feel. For example, one chilly night I was on a mountain hike with some students. I told them that we were going to have to cross a mountain stream. They began to grumble about how cold it would be. We reached the stream and they reluctantly plunged ahead. They were almost up to their knees in the water when they realized that I had led them into a hot spring. Later they all said that they had felt cold water at first. We also need to notice signs which can help us see more. Once I was on a hike, following about six meters behind my Indian friend named John. As we passed under a huge pine tree. he turned around and said, "Don't disturb it." Surprised, I looked everywhere. Was there a deer, a fox, or something else that I had missed seeing? Finally I looked into the branches of the tree and saw a beautiful owl not three meters from us. I was amazed that Stalking Wolf had known it was there without looking up. When I asked him how he had known the owl was there, he replied, "Go ask the mice." By looking down he had seen the tracks of the mice that had run away from their dreaded enemy, the owl. The next time you take a walk, no matter where it is, open up your eyes. Be aware of all the sights, sounds and sensations. You will be surprised at the many beautiful and interesting things you have been missing. """

In [ ]:
text_br = text_br.replace("Here's", "Here is");   text_br = text_br.replace("here's", "here is");   text_br = text_br.replace("I'm", "I am");   text_br = text_br.replace("I'd", "I would");   text_br = text_br.replace("I've", "I have");   text_br = text_br.replace("I'll", "I will");    text_br = text_br.replace("that's", "that is");   text_br = text_br.replace("That's", "That is");     text_br = text_br.replace("What's", "What is");   text_br = text_br.replace("You'll", "You will");   text_br = text_br.replace("you'll", "you will");  text_br = text_br.replace("It's", "It is");   text_br = text_br.replace("it's", "it is");  text_br = text_br.replace("It'll", "It will");   text_br = text_br.replace("it'll", "it will");  text_br = text_br.replace("You're", "You are");  text_br = text_br.replace("you're", "you are");   text_br = text_br.replace("You've", "You have");  text_br = text_br.replace("you've", "you have");   text_br = text_br.replace("You've", "You have");  
text_br = text_br.replace("We'll", "We will");   text_br = text_br.replace("we'll", "we will");  text_br = text_br.replace("She'll", "She will");   text_br = text_br.replace("she'll", "she will");  text_br = text_br.replace("He'll", "He will");  text_br = text_br.replace("he'll", "he will");  text_br = text_br.replace("They'll", "They will");  text_br = text_br.replace("they'll", "they will");   text_br = text_br.replace("They're", "They are");  text_br = text_br.replace("they're", "they are");  text_br = text_br.replace("They've", "They have");  text_br = text_br.replace("they've", "they have"); 
text_br = text_br.replace("don't", "do not");   text_br = text_br.replace("doesn't", "does not");   text_br = text_br.replace("didn't", "did not");   text_br = text_br.replace("isn't", "is not");   text_br = text_br.replace("wasn't", "was not");   text_br = text_br.replace("aren't", "are not");  text_br = text_br.replace("hasn't", "has not");    text_br = text_br.replace("haven't", "have not");  text_br = text_br.replace("hadn't", "had not");   text_br = text_br.replace("won't", "will not");  
text_br = text_br.replace("can't", "can not");  text_br = text_br.replace("cannot", "can not");  text_br = text_br.replace("shouldn't", "should not");  text_br = text_br.replace("wouldn't", "would not");  text_br = text_br.replace(" Miss ", " dr ");    text_br = text_br.replace(" Pip ", " john ");  
text_br = text_br.replace("'", "");     # for 'bryant-stories.txt'

In [ ]:
text = ''
for i in range(len(text_)):     text = text + text_[i]
text = text + text_br + txt_1

In [ ]:
text = text.replace('\r\n', ' ');  text = text.replace('\n', ' ');   # \n is inserted not at the end of sentence.
text = text.replace("Here's", "Here is");   text = text.replace("here's", "here is");   text = text.replace("I'm", "I am");   text = text.replace("I'd", "I would");   text = text.replace("I've", "I have");   text = text.replace("I'll", "I will");    text = text.replace("that's", "that is");   text = text.replace("That's", "That is");     text = text.replace("What's", "What is"); 
text = text.replace("We'll", "We will");   text = text.replace("we'll", "we will");  text = text.replace("You'll", "You will");   text = text.replace("you'll", "you will");  text = text.replace("It's", "It is");   text = text.replace("it's", "it is");  text = text.replace("It'll", "It will");   text = text.replace("it'll", "it will");  text = text.replace("You're", "You are");  text = text.replace("you're", "you are");   text = text.replace("You've", "You have");  text = text.replace("you've", "you have");   text = text.replace("You've", "You have");  
text = text.replace("She'll", "She will");   text = text.replace("she'll", "she will");  text = text.replace("He'll", "He will");  text = text.replace("he'll", "he will");  text = text.replace("They'll", "They will");  text = text.replace("they'll", "they will");   text = text.replace("They're", "They are");  text = text.replace("they're", "they are");  text = text.replace("They've", "They have");  text = text.replace("they've", "they have"); 
text = text.replace("don't", "do not");   text = text.replace("doesn't", "does not");   text = text.replace("didn't", "did not");   text = text.replace("isn't", "is not");   text = text.replace("wasn't", "was not");   text = text.replace("aren't", "are not");  text = text.replace("hasn't", "has not");    text = text.replace("haven't", "have not");  text = text.replace("hadn't", "had not");   text = text.replace("won't", "will not");  text = text.replace("can't", "can not");  text = text.replace("cannot", "can not");  text = text.replace("shouldn't", "should not");  text = text.replace("wouldn't", "would not");
text = text.replace("gonna", "going to");   text = text.replace("wanna", "want to");   text = text.replace(" Miss ", " dr ");   text = text.replace(" Pip ", " john ");
# text = text.replace("'", "");  # for 'bryant-stories.txt'

In [ ]:
nltk.download('punkt')
text = nltk.tokenize.sent_tokenize(text) ## generating sentenses

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
##### preprocessing: selecting sentences #####
#sents1 = [];
#for line in range(len(text)):
#    words = text[line].split()    # text[line] will be converted to lists of words
#    if (len(words) >= 3):
#        sents1.append(text[line])

In [ ]:
##### preprocessing: selecting sentences #####
import re 
sents1 = [];
for line in range(len(text)):
    words = text[line].split()    # text[line] will be converted to lists of words
    if (len(words) >= 3):
        line_s = re.split(r'[:;]',text[line]);
        if len(line_s)>1:        ## finding multiple sentenses in one line
            # line_s = re.split(r'.|;|:',text[line]);
            for i in range(len(line_s)):    
                if len(line_s[i])>4:    sents1.append(line_s[i])
        else:  sents1.append(text[line]);

In [ ]:
# len(')')

In [ ]:
sents1[:120]

["[Alice's Adventures in Wonderland by Lewis Carroll 1865]  CHAPTER I.",
 'Down the Rabbit-Hole  Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do',
 " once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, 'and what is the use of a book,' thought Alice 'without pictures or conversation?'",
 'So she was considering in her own mind (as well as she could, for the hot day made her feel very sleepy and stupid), whether the pleasure of making a daisy-chain would be worth the trouble of getting up and picking the daisies, when suddenly a White Rabbit with pink eyes ran close by her.',
 'There was nothing so VERY remarkable in that',
 " nor did Alice think it so VERY much out of the way to hear the Rabbit say to itself, 'Oh dear!",
 "I shall be late!'",
 '(when she thought it over afterwards, it occurred to her that she ought to have wondered at this, but at the time it all seemed 

In [ ]:
# nltk.corpus.brown.categories()

In [ ]:
nltk.download('brown')
text_b,sents2=[],[]
text_b.append( nltk.corpus.brown.sents(categories='adventure') )
text_b.append( nltk.corpus.brown.sents(categories='news') )
text_b.append( nltk.corpus.brown.sents(categories='belles_lettres') )
text_b.append( nltk.corpus.brown.sents(categories='humor') )
text_b.append( nltk.corpus.brown.sents(categories='hobbies') )
text_b.append( nltk.corpus.brown.sents(categories='reviews') )
text_b.append( nltk.corpus.brown.sents(categories='romance') )
text_b.append( nltk.corpus.brown.sents(categories='science_fiction') )
text_b.append( nltk.corpus.brown.sents(categories='mystery') )
text_b.append( nltk.corpus.brown.sents(categories='fiction') )

for i in range(len(text_b)): 
    for ln in range(len(text_b[i])):
        line = ' '.join(text_b[i][ln]) ## text_b is lists of words (in brown corpus) 
        line = line.replace("Here's", "Here is");   line = line.replace("here's", "here is");   line = line.replace("I'm", "I am");   line = line.replace("I'd", "I would");   line = line.replace("I've", "I have");   line = line.replace("I'll", "I will");    line = line.replace("that's", "that is");   line = line.replace("That's", "That is");     line = line.replace("What's", "What is");   line = line.replace("there's", "there is");   line = line.replace("There's", "There is");
        line = line.replace("We'll", "We will");   line = line.replace("we'll", "we will");  line = line.replace("You'll", "You will");   line = line.replace("you'll", "you will");  line = line.replace("It's", "It is");   line = line.replace("it's", "it is");  line = line.replace("It'll", "It will");   line = line.replace("it'll", "it will");  line = line.replace("You're", "You are");  line = line.replace("you're", "you are");   line = line.replace("You've", "You have");  line = line.replace("you've", "you have");   line = line.replace("You've", "You have");  
        line = line.replace("She'll", "She will");   line = line.replace("she'll", "she will");  line = line.replace("He'll", "He will");  line = line.replace("he'll", "he will");  line = line.replace("They'll", "They will");  line = line.replace("they'll", "they will");   line = line.replace("They're", "They are");  line = line.replace("they're", "they are");  line = line.replace("They've", "They have");  line = line.replace("they've", "they have"); 
        line = line.replace("don't", "do not");   line = line.replace("doesn't", "does not");   line = line.replace("didn't", "did not");   line = line.replace("isn't", "is not");   line = line.replace("wasn't", "was not");   line = line.replace("aren't", "are not");  line = line.replace("hasn't", "has not");    line = line.replace("haven't", "have not");  line = line.replace("hadn't", "had not");   line = line.replace("won't", "will not");  line = line.replace("can't", "can not");  line = line.replace("cannot", "can not");  line = line.replace("shouldn't", "should not");  line = line.replace("couldn't", "could not");  line = line.replace("wouldn't", "would not");
        line = line.replace("gonna", "going to");   line = line.replace("wanna", "want to");     line = line.replace(" Miss ", " dr ");   line = line.replace(" Pip ", " john ");
        sents2.append(line)

sents = sents2 + sents1

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [ ]:
text_b[0]

[['Dan', 'Morgan', 'told', 'himself', 'he', 'would', 'forget', 'Ann', 'Turner', '.'], ['He', 'was', 'well', 'rid', 'of', 'her', '.'], ...]

In [ ]:
sents1[-19:]

['For example, one chilly night I was on a mountain hike with some students.',
 'I told them that we were going to have to cross a mountain stream.',
 'They began to grumble about how cold it would be.',
 'We reached the stream and they reluctantly plunged ahead.',
 'They were almost up to their knees in the water when they realized that I had led them into a hot spring.',
 'Later they all said that they had felt cold water at first.',
 'We also need to notice signs which can help us see more.',
 'Once I was on a hike, following about six meters behind my Indian friend named John.',
 'As we passed under a huge pine tree.',
 'he turned around and said, "Don\'t disturb it."',
 'Surprised, I looked everywhere.',
 'Was there a deer, a fox, or something else that I had missed seeing?',
 'Finally I looked into the branches of the tree and saw a beautiful owl not three meters from us.',
 'I was amazed that Stalking Wolf had known it was there without looking up.',
 'When I asked him how he ha

In [ ]:
sents2[24000:24020]

['Sooner or later , all the gray Navy ships came in here ; ;',
 'if Tommy sat long enough , she would be sure to see all the young officers she had met in San Diego and Long Beach .',
 'And she wanted desperately to see someone she had known back there .',
 'She felt , rather than saw , the approach of the good-looking young man .',
 'He came through from the Fleet Bar , which was stag , with the ice cubes tinkling in a glass he carried .',
 'When he saw Tommy sitting alone , the tinkling sound stopped .',
 'He was perhaps a trifle tipsy , having been long at sea where drinking is not permitted , and consequently out of practice ; ;',
 'he wore a brown tweed sports jacket obviously tailored in Hong Kong , and he was of an age that marked him as a lieutenant .',
 'Probably off one of the carriers -- an aviator .',
 'There was a fifty-fifty chance , perhaps , that he would be unmarried , and an even more slender chance that his approach would be different .',
 'Japan did something to a m

In [ ]:
######## remove text within parentheses & Number Replacement ############
# import re 
# for i in range(len(sents)):     sents[i] = re.sub(r"\([^()]*\)", "", sents[i])
for i in range(len(sents)):     
    sents[i] = re.sub("[\(\[].*?[\)\]]", "", sents[i])
    sents[i] = re.sub(r"[0-9]+", "0", sents[i])

In [ ]:
for i in range(len(sents)): 
    if (type(sents[i]) != str):  continue
    sents[i] = sents[i].lower()
    sents[i] = ' ' + sents[i]
    sents[i] = sents[i].replace("'s", "s ");  sents[i] = sents[i].replace("'twas", " it was ");   
    sents[i] = sents[i].replace("have'n't", " have not ");   sents[i] = sents[i].replace("ev'n", " even "); 
    sents[i] = sents[i].replace("o'er", " over ");   sents[i] = sents[i].replace("sha'n't", " shall not "); 
    sents[i] = sents[i].replace("""'""", ""); 
    sents[i] = sents[i].replace(".", " "); sents[i] = sents[i].replace("?", " ");  sents[i] = sents[i].replace("!", " ");    sents[i] = sents[i].replace(""" " """, """ """);  sents[i] = sents[i].replace("&", " and "); 
    sents[i] = sents[i].replace('''"''', ' " ');
    sents[i] = sents[i].replace("*", " ");  sents[i] = sents[i].replace("_", " ");sents[i] = sents[i].replace("`", " ");  sents[i] = sents[i].replace(",", " ");  sents[i] = sents[i].replace(".", " ");   sents[i] = sents[i].replace(";", " ");  sents[i] = sents[i].replace(":", " ");  sents[i] = sents[i].replace("?", " ");  sents[i] = sents[i].replace("!", " ");    
    sents[i] = sents[i].replace("&", " and "); sents[i] = sents[i].replace('''"''', ' " ');   sents[i] = sents[i].replace(""" " " """, """ " """);  

    sents[i] = sents[i].replace("- -", "--");  sents[i] = sents[i].replace("----", "--");  sents[i] = sents[i].replace("--", " -- ");  
    sents[i] = sents[i].replace(" 0-", " 0 ");  sents[i] = sents[i].replace("0", " 0 ");  sents[i] = sents[i].replace("  ", " ");  sents[i] = sents[i].replace("  ", " ");  sents[i] = sents[i].replace(" 0 d ", " 0 ");  sents[i] = sents[i].replace(" 0 s ", " 0 ");  sents[i] = sents[i].replace(" 0 st ", " 0 ");  sents[i] = sents[i].replace(" 0 th ", " 0 ");  sents[i] = sents[i].replace(" 0 0 ", " 0 ");  sents[i] = sents[i].replace(" 0 0 ", " 0 ");  sents[i] = sents[i].replace(" 0 0 ", " 0 ");  sents[i] = sents[i].replace(" $ 0 0 ", " 0 dollar ");  sents[i] = sents[i].replace(" $ 0 ", " 0 dollar ");  
    sents[i] = sents[i].replace("   ", " ");  sents[i] = sents[i].replace("   ", " ");  sents[i] = sents[i].replace("   ", " ");  sents[i] = sents[i].replace(" 0 ", " zero ");
    sents[i] = sents[i].replace(" b ", " ");   sents[i] = sents[i].replace(" c ", " ");  sents[i] = sents[i].replace(" d ", " ");  sents[i] = sents[i].replace(" e ", " ");  sents[i] = sents[i].replace(" f ", " ");  sents[i] = sents[i].replace(" g ", " ");  sents[i] = sents[i].replace(" h ", " ");  sents[i] = sents[i].replace(" j ", " ");  sents[i] = sents[i].replace(" k ", " ");  sents[i] = sents[i].replace(" l ", " ");  sents[i] = sents[i].replace(" m ", " ");   sents[i] = sents[i].replace(" n ", " ");   sents[i] = sents[i].replace(" o ", " ");   sents[i] = sents[i].replace(" p ", " ");  sents[i] = sents[i].replace(" q ", " ");  sents[i] = sents[i].replace(" r ", " ");  sents[i] = sents[i].replace(" s ", " ");   sents[i] = sents[i].replace(" t ", " ");  sents[i] = sents[i].replace(" u ", " ");   sents[i] = sents[i].replace(" v ", " ");  sents[i] = sents[i].replace(" w ", " ");  sents[i] = sents[i].replace(" z ", " ");  
    
    sents[i] = sents[i].replace(" pre-", " pre"); sents[i] = sents[i].replace(" mama ", " mother "); sents[i] = sents[i].replace(" em ", " them ");  sents[i] = sents[i].replace(" morrow ", " tomorrow "); sents[i] = sents[i].replace(" to-morrow", " tomorrow")
    sents[i] = sents[i].replace(" de ", "  ");  sents[i] = sents[i].replace(" don ", "  ");  sents[i] = sents[i].replace(" jr ", "  ");  sents[i] = sents[i].replace(" mr ", " dr ");  sents[i] = sents[i].replace(" mrs ", " dr ");  sents[i] = sents[i].replace(" sir ", " dr ");  sents[i] = sents[i].replace(" st ", " dr ");  sents[i] = sents[i].replace(" ah ", "  ");  sents[i] = sents[i].replace(" anneounced ", " announced ");  sents[i] = sents[i].replace(" anti-", " anti");  sents[i] = sents[i].replace(" co ", " cooperation ");  sents[i] = sents[i].replace(" damn it ", " damnit ");  sents[i] = sents[i].replace(" damn war ", " war ");  sents[i] = sents[i].replace(" good bye ", " goodbye ");  sents[i] = sents[i].replace(" hey ", " hi ");  sents[i] = sents[i].replace(" o ", " of ");  sents[i] = sents[i].replace(" oh ", "  ");  sents[i] = sents[i].replace(" re-", " re");  sents[i] = sents[i].replace(" san ", " san");  sents[i] = sents[i].replace(" sperm whale", " whale");   
    sents[i] = sents[i].replace(" aint ", " are not ");  sents[i] = sents[i].replace(" aj ", "  ");  sents[i] = sents[i].replace(" ann ", " anne ");   sents[i] = sents[i].replace(" ay ", " yes ");  sents[i] = sents[i].replace(" aye ", " yes ");  sents[i] = sents[i].replace(" by the bye ", "  ");  sents[i] = sents[i].replace(" couldnt ", " could not ");  sents[i] = sents[i].replace(" damndue to ", " because of ");  sents[i] = sents[i].replace(" dont ", " do not ");  sents[i] = sents[i].replace(" em  ", " them ");  sents[i] = sents[i].replace(" ere ", " before ");  sents[i] = sents[i].replace(" hitherto ", " until now ");  sents[i] = sents[i].replace(" nay ", " no "); sents[i] = sents[i].replace(" nigh ", " near ");  sents[i] = sents[i].replace(" re ", " about ");   sents[i] = sents[i].replace(" till ", " until ");  sents[i] = sents[i].replace(" tis ", " it is ");  sents[i] = sents[i].replace(" whats ", " what is ");  sents[i] = sents[i].replace(" ye ", " you ");  sents[i] = sents[i].replace(" yes aye ", "  ");  sents[i] = sents[i].replace(" your majesty ", " you ");  sents[i] = sents[i].replace(" thou canst ", " you can ");  sents[i] = sents[i].replace(" thou hast ", " you have ");  sents[i] = sents[i].replace(" thee ", " you ");  sents[i] = sents[i].replace(" thou ", " you ");  sents[i] = sents[i].replace(" thoust ", " you have ");  sents[i] = sents[i].replace(" thy ", " your ");  
    sents[i] = sents[i].replace(" hed be ", " he would be ");  sents[i] = sents[i].replace(" hed been ", " he had been ");  sents[i] = sents[i].replace(" hed come ", " he would come ");  sents[i] = sents[i].replace(" hed creep ", " he would  creep ");  sents[i] = sents[i].replace(" hed decided ", " he had decided ");  sents[i] = sents[i].replace(" hed do ", " he would  do ");  sents[i] = sents[i].replace(" hed give ", " he would give ");  sents[i] = sents[i].replace(" hed got ", " he had got ");  sents[i] = sents[i].replace(" hed had ", " he had had ");  sents[i] = sents[i].replace(" hed have ", " he would have ");  sents[i] = sents[i].replace(" hed leave ", " he would leave ");  sents[i] = sents[i].replace(" hed left ", " he had left ");  sents[i] = sents[i].replace(" hed meet ", " he would meet ");  sents[i] = sents[i].replace(" hed not care ", " he would not care ");  sents[i] = sents[i].replace(" hed take ", " he would take ");  sents[i] = sents[i].replace(" hed tell ", " he would tell ");  sents[i] = sents[i].replace(" hed told ", " he had told ");  sents[i] = sents[i].replace(" youd decided ", " you had decided ");  sents[i] = sents[i].replace(" youd learned ", " you had learned ");  sents[i] = sents[i].replace(" youd missed ", " you had missed ");  
    sents[i] = sents[i].replace(" abra ", " smith ");  sents[i] = sents[i].replace(" ada ", " johnson ");  sents[i] = sents[i].replace(" adams ", " smith ");  sents[i] = sents[i].replace(" baker ", " smith ");  sents[i] = sents[i].replace(" barton ", " smith ");  sents[i] = sents[i].replace(" batess ", " williams ");  sents[i] = sents[i].replace(" benwick ", " williams ");  sents[i] = sents[i].replace(" bildad ", " smith ");  sents[i] = sents[i].replace(" burke ", " williams ");  sents[i] = sents[i].replace(" bursal ", " williams ");  sents[i] = sents[i].replace(" captain harville ", " johnson ");  sents[i] = sents[i].replace(" captain wentworth ", " johnson ");  sents[i] = sents[i].replace(" carlo ", " johnson ");  sents[i] = sents[i].replace(" celies ", " johnson ");  sents[i] = sents[i].replace(" clayton ", " williams ");  sents[i] = sents[i].replace(" dashwood ", " smith ");  sents[i] = sents[i].replace(" dashwoods ", " johnson ");  sents[i] = sents[i].replace(" donwell ", " williams ");  sents[i] = sents[i].replace(" doyle ", " williams ");  sents[i] = sents[i].replace(" edwards ", " williams ");  sents[i] = sents[i].replace(" eltons ", " smith ");  sents[i] = sents[i].replace(" estherson ", " johnson ");  sents[i] = sents[i].replace(" fairfax ", " williams ");  sents[i] = sents[i].replace(" felix ", " williams ");  sents[i] = sents[i].replace(" frederick ", " johnson ");  sents[i] = sents[i].replace(" garland ", " smith ");  sents[i] = sents[i].replace(" gresham ", " johnson ");  sents[i] = sents[i].replace(" grey ", " williams ");  sents[i] = sents[i].replace(" hardwick ", " smith ");  sents[i] = sents[i].replace(" hartfield ", " johnson ");  sents[i] = sents[i].replace(" jennings ", " smith ");  sents[i] = sents[i].replace(" jones ", " smith ");  sents[i] = sents[i].replace(" joseph ", " smith ");  sents[i] = sents[i].replace(" kennedy ", " johnson ");  sents[i] = sents[i].replace(" kiz ", " smith ");  sents[i] = sents[i].replace(" kizzie ", " johnson ");  sents[i] = sents[i].replace(" knightley ", " johnson ");  sents[i] = sents[i].replace(" langford ", " williams ");  sents[i] = sents[i].replace(" lee ", " smith ");  sents[i] = sents[i].replace(" lewis ", " smith ");  sents[i] = sents[i].replace(" loveit ", " williams ");  sents[i] = sents[i].replace(" martin ", " williams ");  sents[i] = sents[i].replace(" maude ", " smith ");  sents[i] = sents[i].replace(" middleton ", " smith ");  sents[i] = sents[i].replace(" miss johnson ", " dr johnson ");  sents[i] = sents[i].replace(" miss smith ", " dr smith ");  sents[i] = sents[i].replace(" miss williams ", " dr williams ");  sents[i] = sents[i].replace(" morgan ", " johnson ");  sents[i] = sents[i].replace(" murphy ", " johnson ");  sents[i] = sents[i].replace(" musgrove ", " williams ");  sents[i] = sents[i].replace(" musgroves ", " williams ");  sents[i] = sents[i].replace(" norland ", " williams ");  sents[i] = sents[i].replace(" palmer ", " johnson ");  sents[i] = sents[i].replace(" peleg ", " johnson ");  sents[i] = sents[i].replace(" piedro ", " smith ");  sents[i] = sents[i].replace(" pulova ", " smith ");  sents[i] = sents[i].replace(" queequeg ", " williams ");  sents[i] = sents[i].replace(" roberts ", " johnson ");  sents[i] = sents[i].replace(" rossoff ", " williams ");  sents[i] = sents[i].replace(" russell ", " johnson ");  sents[i] = sents[i].replace(" schwarzkopf ", " johnson ");  sents[i] = sents[i].replace(" shirley ", " williams ");  sents[i] = sents[i].replace(" stuart ", " johnson ");  sents[i] = sents[i].replace(" sutherland ", " williams ");  sents[i] = sents[i].replace(" talbot ", " williams ");  sents[i] = sents[i].replace(" tamiris ", " smith ");  sents[i] = sents[i].replace(" tarlton ", " johnson ");  sents[i] = sents[i].replace(" taylor ", " smith ");  sents[i] = sents[i].replace(" vaughan ", " johnson ");  sents[i] = sents[i].replace(" vieth ", " williams ");  sents[i] = sents[i].replace(" walter ", " williams ");  sents[i] = sents[i].replace(" watson ", " johnson ");  sents[i] = sents[i].replace(" wentworth ", " williams ");  sents[i] = sents[i].replace(" weston ", " smith ");  sents[i] = sents[i].replace(" westons ", " smiths ");  sents[i] = sents[i].replace(" wheeler ", " smith ");  sents[i] = sents[i].replace(" willoughby ", " johnson ");  sents[i] = sents[i].replace(" wilson ", " williams ");  sents[i] = sents[i].replace(" winston ", " williams ");  sents[i] = sents[i].replace(" woodhouse ", " smith ");  sents[i] = sents[i].replace(" xydis ", " johnson ");  
    sents[i] = sents[i].replace(" annee ", " jane ");  sents[i] = sents[i].replace(" annees ", " janes ");  sents[i] = sents[i].replace(" barbara ", " jane ");  sents[i] = sents[i].replace(" bates ", " emma ");  sents[i] = sents[i].replace(" betsy ", " alice ");  sents[i] = sents[i].replace(" cecilia ", " alice ");  sents[i] = sents[i].replace(" eileen ", " emma ");  sents[i] = sents[i].replace(" elinor ", " jane ");  sents[i] = sents[i].replace(" elinors ", " janes ");  sents[i] = sents[i].replace(" folly ", " emma ");  sents[i] = sents[i].replace(" glenda ", " alice ");  sents[i] = sents[i].replace(" harriet ", " jane ");  sents[i] = sents[i].replace(" harriets ", " janes ");  sents[i] = sents[i].replace(" henrietta ", " jane ");  sents[i] = sents[i].replace(" isabella ", " emma ");  sents[i] = sents[i].replace(" jen ", " jane ");  sents[i] = sents[i].replace(" joan ", " alice ");  sents[i] = sents[i].replace(" julia ", " emma ");  sents[i] = sents[i].replace(" kate ", " jane ");  sents[i] = sents[i].replace(" katherine ", " alice ");  sents[i] = sents[i].replace(" laura ", " emma ");  sents[i] = sents[i].replace(" leonora ", " alice ");  sents[i] = sents[i].replace(" linda ", " jane ");  sents[i] = sents[i].replace(" louisa ", " jane ");  sents[i] = sents[i].replace(" margaret ", " alice ");  sents[i] = sents[i].replace(" mariannee ", " alice ");  sents[i] = sents[i].replace(" mariannee ", " jane ");  sents[i] = sents[i].replace(" mariannees ", " janes ");  sents[i] = sents[i].replace(" mary ", " jane ");  sents[i] = sents[i].replace(" miss alice ", " dr alice ");  sents[i] = sents[i].replace(" miss emma ", " dr emma ");  sents[i] = sents[i].replace(" miss jane ", " dr jane ");  sents[i] = sents[i].replace(" rachel ", " alice ");  sents[i] = sents[i].replace(" rosamond ", " alice ");  sents[i] = sents[i].replace(" sophy ", " alice ");  sents[i] = sents[i].replace(" susan ", " emma ");  sents[i] = sents[i].replace(" susans ", " emmas ");  sents[i] = sents[i].replace(" theresa ", " emma ");  
    sents[i] = sents[i].replace(" ahab ", " elliot ");  sents[i] = sents[i].replace(" ahabs ", " elliots ");  sents[i] = sents[i].replace(" ahalexander ", " elton ");  sents[i] = sents[i].replace(" ahtom ", " elliot ");  sents[i] = sents[i].replace(" alfred ", " elliot ");  sents[i] = sents[i].replace(" bildad ", " elton ");  sents[i] = sents[i].replace(" billy ", " edward ");  sents[i] = sents[i].replace(" captain benwick ", " edward ");  sents[i] = sents[i].replace(" captain bildad ", " elton ");  sents[i] = sents[i].replace(" captain peleg ", " elliot ");  sents[i] = sents[i].replace(" charles ", " elton ");  sents[i] = sents[i].replace(" charlie ", " elton ");  sents[i] = sents[i].replace(" churchill ", " elliot ");  sents[i] = sents[i].replace(" david ", " elton ");  sents[i] = sents[i].replace(" francisco ", " edward ");  sents[i] = sents[i].replace(" franklin ", " edward ");  sents[i] = sents[i].replace(" george ", " elton ");  sents[i] = sents[i].replace(" hal ", " elton ");  sents[i] = sents[i].replace(" hans ", " elliot ");  sents[i] = sents[i].replace(" hartfield ", " smith ");  sents[i] = sents[i].replace(" jack ", " edward ");  sents[i] = sents[i].replace(" james ", " edward ");  sents[i] = sents[i].replace(" jem ", " elliot ");  sents[i] = sents[i].replace(" jonah ", " elliot ");  sents[i] = sents[i].replace(" lawrence ", " elton ");  sents[i] = sents[i].replace(" louis ", " edward ");  sents[i] = sents[i].replace(" mike ", " elton ");  sents[i] = sents[i].replace(" nayopeleg ", " elliot ");  sents[i] = sents[i].replace(" paul ", " edward ");  sents[i] = sents[i].replace(" perry ", " elton ");  sents[i] = sents[i].replace(" peter ", " eliot ");  sents[i] = sents[i].replace(" phil ", " elliot ");  sents[i] = sents[i].replace(" philip ", " edward ");  sents[i] = sents[i].replace(" richard ", " elton ");  sents[i] = sents[i].replace(" robert ", " elliot ");  sents[i] = sents[i].replace(" sam ", " edward ");  sents[i] = sents[i].replace(" sammy ", " edward ");  sents[i] = sents[i].replace(" starbuck ", " elton ");  sents[i] = sents[i].replace(" tashtego ", " elliot ");  sents[i] = sents[i].replace(" the hatter ", " eliot ");  sents[i] = sents[i].replace(" the tailor ", " elton ");  sents[i] = sents[i].replace(" thomas ", " elliot ");  sents[i] = sents[i].replace("  ", " ");  
    sents[i] = sents[i].replace(" brahmin ", " man ");  sents[i] = sents[i].replace(" britain ", " england ");  sents[i] = sents[i].replace(" british ", " english ");  sents[i] = sents[i].replace(" dormouse ", " mouse ");  sents[i] = sents[i].replace(" gingerbread boy ", " gingerbreadboy ");  sents[i] = sents[i].replace(" guinea-", "guinea");  sents[i] = sents[i].replace(" guinea pig ", " guineapig ");  sents[i] = sents[i].replace(" guineahatter ", " man ");  sents[i] = sents[i].replace(" harpooneer ", " man ");  sents[i] = sents[i].replace(" harpooneers ", " men ");  sents[i] = sents[i].replace(" highbury ", " paris ");  sents[i] = sents[i].replace(" leviathan ", " monster ");  sents[i] = sents[i].replace(" little fir tree ", " little tree ");  sents[i] = sents[i].replace(" look-outs ", " lookouts ");  sents[i] = sents[i].replace(" maam ", " madam ");  sents[i] = sents[i].replace(" mamma ", " mother ");  sents[i] = sents[i].replace(" mast-heads ", " mastheads ");  sents[i] = sents[i].replace(" moby dick ", " the whale ");  sents[i] = sents[i].replace(" morrow ", " tomorrow ");  sents[i] = sents[i].replace(" new guinea ", " newguinea ");  sents[i] = sents[i].replace(" papa ", " father ");  sents[i] = sents[i].replace(" pequods ", " ships ");  sents[i] = sents[i].replace(" pequods boats ", " ships ");  sents[i] = sents[i].replace(" randalls ", " paris ");  sents[i] = sents[i].replace(" san francisco ", " sanfrancisco ");  sents[i] = sents[i].replace(" seamen ", " men ");  sents[i] = sents[i].replace(" soviet union ", " soviet ");  sents[i] = sents[i].replace(" the guinea ", " the dollar ");  sents[i] = sents[i].replace(" the pequod ", " the ship ");  sents[i] = sents[i].replace(" whaleman ", " man ");  sents[i] = sents[i].replace(" whalemen ", " men ");  sents[i] = sents[i].replace(" young gentleman ", " young man ");  


#text = text.replace("we've", "we have");  
#text = text.replace("we'd", "We would");  


In [ ]:
sents[10000:10001]

[' historians have traditionally regarded the great debates of the seventeen nineties as polarizing the issues of centralized vs limited government with hamilton and the nationalists supporting the former and jefferson and madison upholding the latter position ']

In [ ]:
#### removing chapter header ###
for i in range(len(sents)): 
     if (type(sents[i]) != str):  continue
     if sents[i][:8].find('chapter') != -1:  sents[i] = "";
     elif sents[i].find(') chapter ') != -1:  sents[i] = "";
     elif sents[i][:6].find('page ') != -1:  sents[i] = "";
     elif sents[i].find('[illustration') != -1:  sents[i] = "";

In [ ]:
#### removing afer (" said), (e.g. "I will do" said alice.) ###
for i in range(len(sents)): 
     if (type(sents[i]) != str):  continue
     if sents[i].find('" said ') != -1:  sents[i] = sents[i][:sents[i].find('" said ')];
     # if sents[i].find('" answered ') != -1:  sents[i] = sents[i][:sents[i].find('" answered ')];

In [ ]:
#### removing French sentences ###
for i in range(len(sents)): 
     if (type(sents[i]) != str):  continue
     if sents[i].find(' la ') != -1:  sents[i] = "";
     elif sents[i].find(' non ') != -1:  sents[i] = "";

In [ ]:
#### removing sentences contains hi gentlemen/gentleman sir damn or af ###
for i in range(len(sents)): 
     if (type(sents[i]) != str):  continue
     if sents[i].find(' hi ') != -1:  sents[i] = "";  ## <40 sentences
     elif sents[i].find(' gentlemen ') != -1:  sents[i] = "";
     elif sents[i].find(' a d 0 ') != -1:  sents[i] = "";
     elif sents[i].find(' af ') != -1:  sents[i] = "";
     elif sents[i][:6].find('said ') != -1:  sents[i] = "";
    #  elif sents[i].find(' damn ') != -1:  sents[i] = "";

In [ ]:
for i in range(len(sents)): 
     if (type(sents[i]) != str):  continue
     if sents[i].find("pre-") != -1:    print(sents[i])

In [ ]:
for i in range(len(sents)): 
     if (type(sents[i]) != str):  continue
     if sents[i][:3].find(' hole ') != -1:    print(sents[i])

In [ ]:
for i in range(len(sents)): 
     if (type(sents[i]) != str):  continue
     if sents[i].find(' whale ') != -1:    print(sents[i])

 -- websters dictionary " whale 
 therefore you must not in every case at least take the higgledy-piggledy whale statements however authentic in these extracts for veritable gospel cetology 
 " touching that monstrous bulk of the whale or ork we have received nothing certain 
 they grow exceeding fat insomuch that an incredible quantity of oil will be extracted out of one whale " 
 " very like a whale " 
 " which to secure no skill of leachs art mote him availle but to returne againe to his wounds worker that with lowly dart dinting his breast had bred his restless paine like as the wounded whale to shore flies thro the maine " 
 of sperma ceti and the sperma ceti whale 
 " silly mansoul swallowed it without chewing as if it had been a sprat in the mouth of a whale " 
 " while the whale is floating at the stern of the ship they cut off his head and tow it with a boat as near the shore as it will come
 some say the whale can not open his mouth but that is a fable 
 they frequently climb

In [ ]:
for i in range(len(sents)): 
     if (type(sents[i]) != str):  continue
     if sents[i].find('`') != -1:    print(sents[i])

In [ ]:
for i in range(len(sents)): 
     if (type(sents[i]) != str):  continue
     if sents[i].find(' hi ') != -1:    print(sents[i])

In [ ]:
for i in range(len(sents)): 
     if (type(sents[i]) != str):  continue
     if sents[i].find("""'""") != -1:    print(sents[i])

In [ ]:
##### preprocessing: tokenizing ######    Note: Tokenizer will replace "-" to " " (e.g. "well-known" -> "well known")  ##
num_words = 3000
tokenizer = preprocessing.text.Tokenizer(oov_token='unk')
# tokenizer = preprocessing.text.Tokenizer(filters='!"#$%&()*+,./:;<=>?@[\\]^_`{|}~\t\n', oov_token='unk') # removed "-" from the filter
tokenizer.fit_on_texts(sents)
tokenizer.word_index = {e:i for e,i in tokenizer.word_index.items() if i <= num_words} ####
tokenizer.word_index[tokenizer.oov_token] = num_words + 1 #### These two lines are necessary to remove rare words 
w_idx = tokenizer.word_index
w_idx['_'] = 0;   w_idx['unk'] = 1;   # w_idx['eats'] = 3000;   
idx_to_word = dict([(value, key) for (key, value) in w_idx.items()])

In [ ]:
print(w_idx)

{'unk': 1, 'the': 2, 'and': 3, 'of': 4, 'to': 5, 'a': 6, 'in': 7, 'was': 8, 'i': 9, 'he': 10, 'it': 11, 'that': 12, 'not': 13, 'his': 14, 'for': 15, 'you': 16, 'is': 17, 'as': 18, 'her': 19, 'with': 20, 'had': 21, 'be': 22, 'she': 23, 'but': 24, 'at': 25, 'on': 26, 'have': 27, 'dr': 28, 'all': 29, 'this': 30, 'him': 31, 'by': 32, 'they': 33, 'from': 34, 'so': 35, 'would': 36, 'were': 37, 'one': 38, 'which': 39, 'no': 40, 'will': 41, 'there': 42, 'are': 43, 'or': 44, 'my': 45, 'could': 46, 'do': 47, 'an': 48, 'what': 49, 'been': 50, 'when': 51, 'zero': 52, 'me': 53, 'their': 54, 'them': 55, 'very': 56, 'if': 57, 'more': 58, 'out': 59, 'who': 60, 'up': 61, 'said': 62, 'we': 63, 'now': 64, 'can': 65, 'little': 66, 'did': 67, 'your': 68, 'than': 69, 'then': 70, 'any': 71, 'some': 72, 'into': 73, 'time': 74, 'about': 75, 'only': 76, 'like': 77, 'has': 78, 'man': 79, 'such': 80, 'must': 81, 'much': 82, 'well': 83, 'other': 84, 'smith': 85, 'good': 86, 'how': 87, 'before': 88, 'know': 89, 'ja

In [ ]:
tokenizer.oov_token

'unk'

In [ ]:
len(w_idx)

3001

In [ ]:
##### time steps of the preceeding words #####
time_step = 16;

In [ ]:
sent_2 = tokenizer.texts_to_sequences(sents);      

In [ ]:
seq_length = 2 * time_step;
sent_2 = preprocessing.sequence.pad_sequences(sent_2, maxlen=seq_length-time_step+1, padding='pre', truncating='post')
sent_2 = np.concatenate( (np.zeros((np.size(sent_2,0),time_step-1)),sent_2), axis=1)

In [ ]:
sents_0 = sent_2+0
np.shape(sents_0)

(88711, 32)

In [ ]:
sent_2 = sent_2[np.sum(sent_2>0,axis=1)>2,:]  # removing too short sentences

In [ ]:
sent_2 = sent_2[np.sum(sent_2[:,0:time_step+12]==w_idx['unk'],axis=1)<4,:]  # unk

In [ ]:
np.shape(sent_2)

(79444, 32)

In [ ]:
np.sum(sent_2==w_idx['read'])

214

In [ ]:
np.sum(sent_2==w_idx['open'])

317

In [ ]:
np.sum(sent_2==w_idx['sail'])

82

In [ ]:
valid = np.sum(sents_0[:,0:time_step+12]==w_idx['unk'],axis=1)<4

In [ ]:
sent_sail = np.nonzero(np.logical_and(valid,np.sum(sents_0==w_idx['sail'],axis=1)>0) )
for i in sent_sail[0]: 
     if (type(sents[i]) != str):  continue
     if sents[i].find(' sail ') != -1:    print(sents[i])

 on the evening that they were to sail smith himself gave a party but he was too indisposed to appear at it 
 you can cross an ocean in a fully equipped craft sail power or both or laze away a fine day in a small dinghy on a local pond 
 fair sail zero 
 the sail was still unfurled though there were a good many holes in it as langer had predicted would be the case by now 
 the sail ragged though it was still had enough surface to catch some of the ocean of power being poured out from the nursery stars 
 a sail on which was to a form a great part of the mornings amusement
 " we set sail from the elbe wind 
 the vessel under short sail with lookouts at the mastheads eagerly scanning the wide expanse around them has a totally different air from those engaged in regular voyage " 
 i love to sail forbidden seas and land on barbarous coasts 
 for my mind was made up to sail in no other than a nantucket craft because there was a fine boisterous something about everything connected with that f

In [ ]:
for i in range(len(sents)): 
     if (type(sents[i]) != str):  continue
     if sents[i].find(' read ') != -1:    print(sents[i])

 i had the impression that he had read my forms perhaps several times 
 dimly he heard laughter hoots of derision but he could not read the racket properly 
 looking back i saw a gray-haired man getting out of his halted car and trying to read our license number 
 if it were not for an old professor who made me read the classics i would have been stymied on what to do and now i understand why they are classics 
 when i was in college i grinned i remember a poem i had to read in my lit class 
 would you first read the poem aloud to me and then let me read it to myself 
 ) but he read on 
 the resolution which barber tossed into the house hopper friday will be formally read monday 
 princess moune prince souvanna phoumas young daughter read the princes statement 
 in opening arthur connelly assistant united states attorney read the indictment but made no comments 
 some zero spectators remained in the court during the day and were on hand to hear the verdict read 
 union oil cooperation 

In [ ]:
for i in range(len(sents)): 
     if (type(sents[i]) != str):  continue
     if sents[i].find(' said ') != -1:    print(sents[i])

 in the kitchen he said 
 we did not want town work smith said 
 this is a mighty empty country johnson said 
 i could use some help johnson said finally but i can not afford to pay you anything 
 we will work for our keep the boy said eagerly 
 i will see johnson said 
 i know smith said dejectedly 
 he said i have got some supper ready 
 i am a mess she said and suddenly she was alarmed 
 she stared at him her eyes wide as she thought about what he had said 
 i do not have many strays coming to my front door he said 
 and hungry he said 
 when they were finally satisfied smith said i think hes going to give us work 
 she said i guess the lord looks out for fools drunkards and innocents 
 were not drunkards she said 
 he said you will feel a lot better after you have a bath 
 no she will be all right smith said quickly 
 we will see johnson said 
 he said i am going to bed 
 you are awfully kind the girl said 
 it is all right he said 
 he said if it is all right with you dr johnson i

In [ ]:
# print(sent_2)